# Follow up answering RAG

In [2]:
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from langchain_community.vectorstores.elasticsearch import ElasticsearchStore
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langsmith.run_helpers import traceable
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

import sys
sys.path.append("../")
from src.config import Configuration
from src.prepare.data_load import DocDataLoader

In [4]:
from operator import itemgetter
from typing import List
from langchain_core.documents import Document

class FollowUpChain:
    def __init__(self, config: Configuration) -> None:
        # chat model
        self.chat_model = ChatGoogleGenerativeAI(
            model="gemini-pro", 
            temperature=0, 
            google_api_key=config.load_gemini_token()
        )
        self._build_chain()
        return

    def _build_chain(self):
        template = """\
            Answer the question based only on the following context. Output "None" if you cannot answer: \
            ```
            {context} \
            ```
            Question: {question}"""
        self.prompt = ChatPromptTemplate.from_template(template)
        self.chain = {
            "context": itemgetter("context"),
            "question": itemgetter("question")
        } | self.prompt | self.chat_model | StrOutputParser()
        return

    def answer(self, docs: List[Document], question: str) -> str:
        ans = ""
        for doc in docs:
            ans = self.chain.invoke({"context": doc.page_content, "question": question})
            if ans.lower() != "none":
                break
        return ans

In [5]:
followup = FollowUpChain(config=Configuration())

In [12]:
db = [Document(page_content="Tom is a 40 years old AI developer"), 
      Document(page_content="John is 23 years old web developer")]
q = "What Tim does"
a = followup.answer(docs=db, question=q)
a

'None'